# Cargar librerias

In [1]:
import pandas as pd
import numpy as np
import openpyxl
import os
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime, timedelta
import warnings

# Cargar bases de datos 

In [2]:
Plano532_COVI = "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_532/Bases/Plano532_COVI.xlsx"
df = pd.read_excel(Plano532_COVI, header=10, skiprows=[11])
df = df.dropna(axis=1, how='all')
df = df[:-2]

In [4]:
# Nueva ruta del archivo consolidado
ruta_excel_covi = "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/Fuentes de Información COVI.xlsx"

# Cargar la hoja RVA_LOCAL desde el archivo consolidado
RVA_LOCAL_df = pd.read_excel(ruta_excel_covi,sheet_name="RVA_LOCAL",usecols="V:AD",nrows=4,header=None)

In [5]:
ruta_excel = "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/Fuentes de Información COGN.xlsx"

BALANCE = pd.read_excel(ruta_excel,sheet_name="BALANCE_WP",usecols="A:E",skiprows=5,dtype=str)
BALANCE.columns = ["CUENTA", "NOMBRE", "VALOR", "VALOR2", "VALOR MONEDA EXTRANJERA"]

# VALIDACIONES 

In [7]:
# Suprimir advertencias específicas
warnings.filterwarnings("ignore", category=FutureWarning)

# Lista de ramos específicos a incluir, en mayúsculas para normalizar
ramos_especificos = [
    "Riesgos Laborales", "Pens. conmutación", "Pens Ley 100", "Previsionales",
    "Accidentes Personales", "Alto Costo", "Vida Individual", "Vida Grupo", "Salud",
    "Exequias", "Colectivo Vida", "Enfermedades de Alto Costo", "Previsional de Invalidez y Sobrevivencia"
]
ramos_especificos = [ramo.upper() for ramo in ramos_especificos]  # Convertir a mayúsculas para normalizar

# Cargar `df` (Plano532_COVI) y filtrar la fila de "Valor reserva técnica de prima no devengada" en "CONCEPTO"
df['CONCEPTO'] = df['CONCEPTO'].str.upper()
valores_reserva_df = df[df['CONCEPTO'] == 'VALOR RESERVA TÉCNICA DE PRIMA NO DEVENGADA']

# Extraer los nombres de ramos y valores de "PPNC - Gross" sin usar `downcast`
ramos_headers = RVA_LOCAL_df.iloc[0].fillna("").astype(str).tolist()
ppnc_gross_values = RVA_LOCAL_df.iloc[3].fillna(0).astype(float).tolist()
ppnc_gross_dict = dict(zip(ramos_headers, ppnc_gross_values))

# Crear un DataFrame vacío para almacenar los resultados
validaciones_df = pd.DataFrame(columns=['Ramo', 'Valor reserva técnica de prima no devengada', 'PPNC - Gross', 'Diferencia'])

# Extraer y calcular los valores de cada ramo
for ramo in ramos_especificos:
    # Obtener el valor de reserva técnica de prima no devengada en `df`
    valor_reserva = valores_reserva_df[ramo].values[0] if ramo in valores_reserva_df.columns else 0

    # Obtener el valor de PPNC - Gross en `ppnc_gross_dict` usando los nombres de ramos
    ppnc_gross = ppnc_gross_dict.get(ramo.upper(), 0)

    # Calcular la diferencia
    diferencia = valor_reserva - ppnc_gross

    # Crear la fila y solo concatenar si tiene datos
    fila_df = pd.DataFrame({
        'Ramo': [ramo],
        'Valor reserva técnica de prima no devengada': [valor_reserva],
        'PPNC - Gross': [ppnc_gross],
        'Diferencia': [diferencia]
    })
    if not fila_df.isnull().all().all():
        validaciones_df = pd.concat([validaciones_df, fila_df], ignore_index=True)

# Calcular totales y añadir una fila de totales al final del DataFrame
totales = {
    'Ramo': 'TOTAL',
    'Valor reserva técnica de prima no devengada': validaciones_df['Valor reserva técnica de prima no devengada'].sum(),
    'PPNC - Gross': validaciones_df['PPNC - Gross'].sum(),
    'Diferencia': validaciones_df['Diferencia'].sum()
}
totales_df = pd.DataFrame([totales])
validaciones_df = pd.concat([validaciones_df, totales_df], ignore_index=True)

# Guardar el total de la columna "PPNC - Gross" para usar en "Valor PPNC - Gross"
ppnc_gross_total = totales['PPNC - Gross']

# Agregar una fila en blanco después del total
fila_en_blanco = pd.DataFrame({
    'Ramo': [''],
    'Valor reserva técnica de prima no devengada': [None],
    'PPNC - Gross': [None],
    'Diferencia': [None]
})
validaciones_df = pd.concat([validaciones_df, fila_en_blanco], ignore_index=True)


# Convertir la columna "VALOR" a numérico y eliminar comas si existen
BALANCE["VALOR"] = pd.to_numeric(BALANCE["VALOR"].astype(str).str.replace(",", ""), errors="coerce")

# Extraer el valor de la cuenta "261000"
cuenta_valor = BALANCE[BALANCE["CUENTA"] == "261000"]["VALOR"].values[0]

# Crear filas adicionales para "CUENTA", "Valor de Cuenta", y "Valor PPNC - Gross"
cuenta_filas = pd.DataFrame({
    'Ramo': ['CUENTA', 'Valor de Cuenta', 'Valor PPNC - Gross'],
    'Valor reserva técnica de prima no devengada': [261000, cuenta_valor, ppnc_gross_total],
    'PPNC - Gross': [None, None, None],
    'Diferencia': [None, None, None]
})

# Agregar `cuenta_filas` al DataFrame final
validaciones_df = pd.concat([validaciones_df, cuenta_filas], ignore_index=True)

# Guardar los resultados en una nueva hoja "VALIDACIONES" en el archivo `Plano532_COVI`
Plano532_COVI = 'D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_532/Bases/Plano532_COVI.xlsx'  # Reemplaza esta ruta según corresponda
with pd.ExcelWriter(Plano532_COVI, mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
    validaciones_df.to_excel(writer, index=False, sheet_name="VALIDACIONES")


## PROFORMA TXT

In [8]:
# Definir las columnas clave
columna_subcuenta = 'SUBCUENTA'
columna_unidad_captura = 'UNIDAD DE CAPTURA'

# Inicializar la lista para almacenar las líneas del archivo TXT
txt_datos = []

# Obtener la fecha correcta (mes vencido)
fecha_actual = datetime.now()
fecha_mes_vencido = fecha_actual.replace(day=1) - timedelta(days=1)
fecha_str = fecha_mes_vencido.strftime("%d%m%Y")  # Formato día-mes-año

# Encabezados fijos de la primera y segunda línea
registro_inicial = f"00001114000001{fecha_str}00033SVIDCOLSECS0184"
segunda_linea = "000022000001100000"

# Agregar las dos primeras líneas a la lista `txt_datos`
txt_datos.append(registro_inicial)
txt_datos.append(segunda_linea)

# Filtrar las filas que no tengan datos válidos en la columna 'SUBCUENTA'
df = df.dropna(subset=[columna_subcuenta])

# Función para formatear la línea de la cuarta parte
def formatear_linea_cuarta_parte(numero_registro, codigo_formato, columna_num, unidad_captura, subcuenta, signo, valor):
    num_registro_str = str(numero_registro).zfill(5)  # Asegurar 5 dígitos para el número de registro
    tipo_registro = "4"  # Tipo de registro siempre debe ser 1 dígito
    codigo_formato_str = str(codigo_formato).zfill(3)  # 3 dígitos para el código del formato
    columna_str = f"{columna_num:02d}"  # Asegurar que el número de columna sea un entero con 2 dígitos
    unidad_captura_str = f"{int(float(unidad_captura)):02d}"  # Convertir a entero y asegurar 2 dígitos para la unidad de captura
    subcuenta_str = f"{int(float(subcuenta)):03d}"  # Convertir a entero y asegurar 3 dígitos para la subcuenta
    valor_str = f"{int(valor):020d}"  # Valor formateado a 20 caracteres con ceros a la izquierda
    return f"{num_registro_str}{tipo_registro}{codigo_formato_str}{columna_str}{unidad_captura_str}{subcuenta_str}{signo}{valor_str}"

# Inicializar el número de registro
numero_registro = 3  # Iniciar en 3 porque las dos primeras líneas ya están fijas

# Recorrer las columnas y filas para generar las líneas, asegurando que el número de columna se incremente correctamente
for columna_num, columna in enumerate(df.columns, start=-1):  # 'start=-1' para iniciar en 01
    if columna not in [columna_subcuenta, columna_unidad_captura]:
        for _, row in df.iterrows():
            subcuenta = row[columna_subcuenta]
            unidad_captura = row[columna_unidad_captura]
            valor = row[columna]

            # Saltar si el valor es cero o NaN
            if pd.isna(valor) or valor == 0:
                continue

            # Determinar el signo del valor
            try:
                valor_float = float(valor)
                signo = '+' if valor_float >= 0 else '-'
                valor_abs = abs(valor_float)

                # Formatear la línea específica para el Formato 532
                linea = formatear_linea_cuarta_parte(numero_registro, 532, columna_num, unidad_captura, subcuenta, signo, valor_abs)
                txt_datos.append(linea)

                numero_registro += 1

            except ValueError:
                # Omitir el mensaje de error
                pass

# Agregar la última línea con el conteo de registros y un "5" al final
total_registros = len(txt_datos) + 1  # Contar todas las líneas generadas
linea_final = f"{str(total_registros).zfill(5)}5"  # Formatear el total con un "5" al final
txt_datos.append(linea_final)

# Guardar el archivo TXT final con todas las partes sin agregar un salto de línea al final
with open("Plano532_VID.txt", "w") as f:
    # Escribir todas las líneas excepto la última con un salto de línea
    for linea in txt_datos[:-1]:
        f.write(f"{linea}\n")
    # Escribir la última línea sin un salto de línea adicional
    f.write(txt_datos[-1])
